In [1]:
from time import sleep
import sys
sys.path.append("/Users/Beniso/Downloads/promethee-develop")
from preference_functions import PreferenceFunctions 
from promethee_simulation import prometheeSimulation;
import utils
from promethee import Promethee

In [2]:
# coding=utf-8
import utils
from preference_functions import PreferenceFunctions 
from promethee_simulation import prometheeSimulation;
import utils
from promethee import Promethee

pref = 'usual'
param = {'p': 1, 'q': 1, 'sigma': 0}
# evaluation_table = [[20,80,200000], [50,150,400000], [100,200,600000], [60,170,200000]]

class Promethee:
	# defines the promethee class that we will use to run simulation
	# evalutionTable (array): table of evaluation of the alternatives w.r.t criteria
	# weights (vector): weights for the criteria
	# preference function : type of preference function given (string)
  # self.evaluation_table = [20,80,200000], [50,150,400000], [100,200,600000]

  def __init__(self, data_sendiri, data_threshold, data_weight):
    self.evaluationTable = data_threshold;
    self.evaluationTable.append(data_sendiri)
    self.weights = data_weight

  def startPromethee(self, preference_function, function_params):
    self.numberAlternatives = len(self.evaluationTable);
    self.numberCriteria = len(self.evaluationTable[0]);
    print ("Before Normalized : {0}".format(self.weights))
    self.normalizeWeights();
    print ("After Normalized : {0}".format(self.weights))
    self.p = function_params['p'];
    self.q = function_params['q'];
    self.sigma = function_params['sigma'];
    self.preference_function = preference_function;
    self.assignShapeFunctions();
    # compute pair wise comparison
    self.computeAllRankings();

  def computeAllRankings(self):
    self.computePairWiseComparisonMatrix();
    self.computepositiveFlow();
    self.computeNegativeFlow();
    self.computeGlobalFlow();
    self.computePositiveMatrix();
    self.computeNegativeMatrix();
    self.computePrometheeIMatrix();
    self.computePrometheeIIMatrix();
    #self.getPrometheeIIRanking();

  def normalizeWeights(self):
    # normalizes the weights given as an input;
    sumW = sum(self.weights);
    for i in range(len(self.weights)):
      self.weights[i] = self.weights[i]/sumW;

  def computepositiveFlow(self):
    phi_plus = [0]*self.numberAlternatives;
    for a in range(self.numberAlternatives):
      sum = 0;
      for b in range(self.numberAlternatives):
        sum += self.pi[a][b];
      phi_plus[a] = sum/(self.numberAlternatives-1);
    self.phi_plus = phi_plus;

  def computeNegativeFlow(self):
    phi_minus = [0]*self.numberAlternatives;
    for a in range(self.numberAlternatives):
      sum = 0;
      for b in range(self.numberAlternatives):
        sum += self.pi[b][a];
      phi_minus[a] = sum/(self.numberAlternatives-1);
    self.phi_minus = phi_minus;

  def computeGlobalFlow(self):
    self.phi_global = [0]*self.numberAlternatives;
    for a in range(self.numberAlternatives):
      self.phi_global[a] = self.phi_plus[a]-self.phi_minus[a];

  def getPrometheeIIRanking(self):
    for x in range(len(self.phi_global)):
      print (self.phi_global[x])
    self.prometheeII = sorted(range(len(self.phi_global)),key = lambda k: self.phi_global[k], reverse= True);
    for x in range(len(self.prometheeII)):
      print (self.prometheeII[x]);

  def getPrometheeIRanking(self):
    print("promethee I")

  def assignShapeFunctions(self):
    self.shapeFunction = [None]*self.numberCriteria;
    availablePreferenceFunctions = PreferenceFunctions(self.p, self.q, self.sigma).get_preference_functions();
    assert(self.preference_function in availablePreferenceFunctions);
    for i in range(self.numberCriteria):
      self.shapeFunction[i] = availablePreferenceFunctions[self.preference_function];

  def getPairwiseComparisonMatrix(self):
    self.computePairWiseComparisonMatrix();
    print ("Calculated Pi : {0}".format(self.pi))
    return self.pi;

  def getPairwiseComparisonMatrix(self):
    self.computePairWiseComparisonMatrix();
    print ("Calculated Pi : (0)".format(self.pi))
    return self.pi;

  def computePairWiseComparisonMatrix(self):
    self.pi = utils.initialise_matrix(self.numberAlternatives,self.numberAlternatives);
    self.default_pi = self.pi;
    print ("Default Pi : {0}".format(self.default_pi))
    for a in range(self.numberAlternatives):
      for b in range(self.numberAlternatives):
        if a!=b:
          sum = 0;
          for k in range(self.numberCriteria):
            delta = self.evaluationTable[a][k]-self.evaluationTable[b][k];
            sum += self.weights[k]*self.shapeFunction[k](delta);
          self.pi[a][b] = sum;
    print ("Calculated Pi : {0}".format(self.pi))


  def computePositiveMatrix(self):
    self.positive_matrix = utils.initialise_matrix(self.numberAlternatives,self.numberAlternatives);
    for a in range(self.numberAlternatives):
      for b in range(self.numberAlternatives):
        if self.phi_plus[a]>self.phi_plus[b]:
          self.positive_matrix[a][b] = 1;
        elif self.phi_plus[a]<self.phi_plus[b]:
          self.positive_matrix[a][b] = -1;
        else:
          self.positive_matrix[a][b] = 0;

  def computeNegativeMatrix(self):
    self.negative_matrix = utils.initialise_matrix(self.numberAlternatives,self.numberAlternatives);
    for a in range(self.numberAlternatives):
      for b in range(self.numberAlternatives):
        if self.phi_minus[a]<self.phi_minus[b]:
          self.negative_matrix[a][b] = 1;
        elif self.phi_minus[a]>self.phi_minus[b]:
          self.negative_matrix[a][b] = -1;
        else:
          self.negative_matrix[a][b] = 0;

  def computePrometheeIMatrix(self):
    self.prometheeI_matrix = utils.initialise_matrix(self.numberAlternatives,self.numberAlternatives);
    for a in range(self.numberAlternatives):
      for b in range(self.numberAlternatives):
        # case 1 : both fluxes give the same result
        # consequently, promethee I ranking is the one given by negative and positive fluxes
        # this covers the sases cas : double preference, double indifferent
        if (self.positive_matrix[a][b] == self.negative_matrix[a][b]):
          self.prometheeI_matrix[a][b] = self.positive_matrix[a][b]
        # case 2: we handle presenfe of an indifference
        else:
          # casa 1: one of the two rankings gives an equality
          # in this case, ranking is the sum of two rankings
          if (self.positive_matrix[a][b]==0 or self.negative_matrix[a][b]==0):
            self.prometheeI_matrix[a][b]= self.positive_matrix[a][b]+self.negative_matrix[a][b];
          # last case: both results are different but none is zero. This gives an incomparability
          else:
            self.prometheeI_matrix[a][b] = "R";

  def computePrometheeIIMatrix(self):
    self.prometheeII_matrix = utils.initialise_matrix(self.numberAlternatives,self.numberAlternatives);
    for a in range(self.numberAlternatives):
      for b in range(self.numberAlternatives):
        if self.phi_plus[a]>self.phi_plus[b]:
          self.prometheeII_matrix[a][b]=1;
        elif self.phi_plus[a]<self.phi_plus[b]:
          self.prometheeII_matrix[a][b]=-1;
        else:
          self.prometheeII_matrix[a][b]=0;

  def getPrometheeIIMatrix(self):
    return self.prometheeII_matrix;

  def getPrometheeIMatrix(self):
    return self.prometheeI_matrix;

  def getPositiveMatrix(self):
    return self.positive_matrix;

  def getNegativeMatrix(self):
    return self.negative_matrix;

  def getPairwiseComparisonMatrix(self):
    return self.pi;

  def getGlobalFlow(self):
    return self.phi_global;


In [3]:
#Ini buat input API

input_data = [200,300,800000,4]
input_weight = [1,1,1,1];
input_threshold = [[20,80,200000,1], [50,150,400000,2], [100,200,600000,3]]

process = Promethee(data_sendiri = input_data, data_threshold = input_threshold, data_weight = input_weight)
process.startPromethee(preference_function=pref, function_params=param)

Before Normalized : [1, 1, 1, 1]
After Normalized : [0.25, 0.25, 0.25, 0.25]
Default Pi : [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
Calculated Pi : [[0, 0.0, 0.0, 0.0], [1.0, 0, 0.0, 0.0], [1.0, 1.0, 0, 0.0], [1.0, 1.0, 1.0, 0]]


In [4]:
last_index = len(process.phi_global)

ranked = sorted(process.phi_global, reverse=False)
for idx, val in enumerate(ranked):
  if process.phi_global[last_index-1] == val:
    output_rank = idx+1
    
output_total = process.phi_global
output_netflow = process.phi_global[last_index-1]


In [5]:
#Ini buat output API

print (output_total)
print (output_netflow)
print (output_rank)

[-1.0, -0.3333333333333333, 0.3333333333333333, 1.0]
1.0
4
